In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# Change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter_jlangley import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "5qjq7q5"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# add in Grazioso Salvare’s logo
image_filename = 'salvare_logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# redefine the original column names to their desired header names
column_mapping = {
    "age_upon_outcome": "Age at Outcome",
    "animal_type": "Animal Type",
    "breed": "Animal Breed",
    "date_of_birth": "Date of Birth",
    "name": "Animal Name",
    "outcome_type": "Outcome Type",
    "sex_upon_outcome": "Sex / Fixed Status",
    "location_lat": "Latitude",
    "location_long": "Longitude",
}

# create a columns list for the data table iterating through all the columns
# but only renaming the ones listed above
columns = []
for column in df.columns:
    if column in column_mapping:
        columns.append({"name": column_mapping[column], "id": column})
    else:
        columns.append({"name": column, "id": column})

app.layout = html.Div([
    # adding the logo and the title of the dash table
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
        children=[
            html.Div(
                style={'display': 'flex', 'align-items': 'center'},
                children=[html.A(
                    html.Img(
                        src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                        alt="Grazio Salvare Logo", 
                        style={'width': '200px', 'height': '200px'}
                    ),
                    href='https://www.snhu.edu'
                ),
                html.Div([
                    html.B(html.H1('CS-340 Project 2')),
                    html.P("author: Joseph Langley")
                ])],
            )
        ]
    ),
    # adding filter buttons
    html.Hr(),
    html.Div(
        children=[
            html.H4('Filter by Rescue Types'),
            html.Div(
                className='buttonRow',
                style={'display': 'flex'},
                children=[
                    dcc.RadioItems(id='filter-type', options=[
                        {'label': 'Water Rescue', 'value': 'wtr'},
                        {'label': 'Mountain Rescue', 'value': 'mtn'},
                        {'label': 'Disaster Rescue', 'value': 'dis'},
                        {'label': 'Reset', 'value': 'res'}
                    ],
                    inline=True)
                ]
            )
        ]
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=columns,
        data=df.to_dict('records'),
        
        # features for your interactive data table to make it user-friendly for your client
        # hides columns that aren't relevant to information needed
        hidden_columns=["rec_num", "animal_id", "color", "datetime", "monthyear", 
                        "outcome_subtype", "age_upon_outcome_in_weeks"],
        # styles for better table appearance
        style_cell={'textAlign': 'left'},
        style_header={
            'backgroundColor': 'rgb(30, 30, 30)',
            'color': 'white'
        },
        style_data={
            'backgroundColor': 'rgb(50, 50, 50)',
            'color': 'white'
        },
        row_selectable="single",
        # adds pages for quicker data retrieval and easier viewing
        page_current=0,
        page_size=5,
        page_action='native',
        # adds sorting functionality
        sort_action="native"
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        className='row',
        style={'display' : 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m4',
            ),
            html.Div(
                id='map-id',
                className='col s12 m8',
            )
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

    # code to filter interactive data table with MongoDB queries
    # columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    # data=df.to_dict('records')
@app.callback(
    Output('datatable-id','data'),
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # display the breeds of animal based on quantity represented in the data table
    # filters for water rescue type dogs, multiple variations based on csv file
    if filter_type == 'wtr':
        data = shelter.read(
            {"breed": {"$in": ["Labrador Retriever Mix", "Chesa Bay Retr Mix", "Newfoundland Mix",
                               "Newfoundland/Labrador Retriever", "Newfoundland/Australian Cattle Dog",
                               "Newfoundland/Great Pyrenees"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        )
    # filters for mountain rescue type dogs
    elif filter_type == 'mtn':
        data = shelter.read(
            {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                               "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        )
    # filters for disaster and individual tracking type dogs
    elif filter_type == 'dis':
        data = shelter.read(
            {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                               "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
        )
    # resets the filter for all results displayed
    else:
        data = shelter.read({})
    
    df = pd.DataFrame.from_records(data)
    df.drop(columns=['_id'],inplace=True)
    
    return df.to_dict('records')

# this callback will create chart and display based on current data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # add code for chart of your choice
    dff = pd.DataFrame.from_dict(viewData)
    
    if viewData is None:
        return []
    
    breeds = dff['breed'].value_counts().keys().tolist() # extracting breed names from viewData
    values = dff['breed'].value_counts().tolist() # counting occurrences of each breed
    
    # create the pie chart
    fig = px.pie(dff, values=values, names=breeds)
    fig.update_traces(textposition = 'inside')
    fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    
    # return the pie chart as a dcc.Graph component
    return [
        dcc.Graph(figure = fig)
    ]

# this callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    if selected_columns is None:
        return[]
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#FFFFFF'
    } for i in selected_columns]

# this callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # code for geolocation chart
    # because we only allow single row selection, the list can be converted to a row index here
    if index is None or viewData is None or len(index) == 0:
        return []
    else:
        row = index[0]
    
    dff = pd.DataFrame.from_dict(viewData)

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1500px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H5("Animal Name:"),
                    html.H5(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)

Dash app running on http://127.0.0.1:19834/
